# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy().drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.8)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler().fit(X_train)

# Fit the scaler to the features training dataset


# Fit the scaler to the features training dataset
X_test_scaled = scaler.transform(X_test)
X_train_scaled = scaler.transform(X_train)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the the number of inputs (features) to the model
inputs = X_train_scaled.shape[1]

# Review the number of features
inputs

11

In [16]:
# Define the number of hidden nodes for the first hidden layer
l1_nodes = 8

# Define the number of hidden nodes for the second hidden layer
l2_nodes = 5

# Define the number of neurons in the output layer
number_output_neurons = 1


In [17]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=l1_nodes, input_dim=inputs, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=l2_nodes, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [18]:
# Display the Sequential model summary
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 96        
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 147 (588.00 Byte)
Trainable params: 147 (588.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [19]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [20]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
40/40 [==============================] - 1s 2ms/step - loss: 0.6763 - accuracy: 0.6099
Epoch 2/50
40/40 [==============================] - 0s 3ms/step - loss: 0.6343 - accuracy: 0.6544
Epoch 3/50
40/40 [==============================] - 0s 2ms/step - loss: 0.6082 - accuracy: 0.6935
Epoch 4/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5883 - accuracy: 0.7138
Epoch 5/50
40/40 [==============================] - 0s 4ms/step - loss: 0.5717 - accuracy: 0.7271
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5590 - accuracy: 0.7349
Epoch 7/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5483 - accuracy: 0.7514
Epoch 8/50
40/40 [==============================] - 0s 1ms/step - loss: 0.5408 - accuracy: 0.7443
Epoch 9/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5350 - accuracy: 0.7435
Epoch 10/50
40/40 [==============================] - 0s 1ms/step - loss: 0.5304 - accuracy: 0.7475
Epoch 11/50
40/40 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [21]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


10/10 - 0s - loss: 0.5209 - accuracy: 0.7563 - 249ms/epoch - 25ms/step
Loss: 0.5209068059921265, Accuracy: 0.7562500238418579


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [22]:
# Set the model's file path
path = Path("saved_models/student_loan_model.keras")

# Export your model to a keras file
nn.save(path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [23]:
# Set the model's file path
path = Path("saved_models/student_loan_model.keras")

# Load the model to a new object
nn_model = tf.keras.models.load_model(path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [24]:
# Make predictions with the test data
predictions = nn_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions[:5]

10/10 [==============================] - 0s 1ms/step


array([[0.47487321],
       [0.2585939 ],
       [0.8677482 ],
       [0.67890847],
       [0.98698854]], dtype=float32)

In [28]:
# Save the predictions to a DataFrame and round the predictions to binary results
results = pd.DataFrame(y_test)
results["predicted"] = predictions.round()

results

,credit_ranking,predicted
75,0,0.0
1283,1,0.0
408,1,1.0
1281,1,1.0
1118,1,1.0
...,...,...
890,0,1.0
146,0,0.0
1551,0,0.0
1209,1,1.0


### Step 4: Display a classification report with the y test data and predictions

In [29]:
# Print the classification report with the y test data and predictions
print(classification_report(results["credit_ranking"], results["predicted"]))

              precision    recall  f1-score   support

           0       0.73      0.79      0.76       154
           1       0.79      0.72      0.75       166

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

